# **Seminar: Politische Debatten & Polarisierung im Bundestag**
## Data Preprocessing Schritte

In diesem Notebook werden vorbereitende Maßnahmen ergriffen, um Ressourcen für die eigentliche Fragestellung der Seminararbeit zu sparen. Das Ziel dieser Schritte ist es am Ende für die Reden aus der 19. und die Reden der 20. Wahlperiode auf Satzebene das jeweilige Sentiment zu erhalten. Für diese Analyse wird unter der Verwendung von Transformer-Modellen das german-sentiment-bert Modell genutzt.Für diese Maßnahmen werden verschiedene individuelle geschriebene Methoden verwendet, die sich im Modul "Sentiment_Methods.py" wiederfinden. 

Abschließend werden die Dataframes mit den Sentiment Daten separat gespeichert und dienen als Grundlage für spätere Analysen. 

## 1. Vorbereitende Maßnahmen

### 1.1 Packages installieren

In [9]:
# Import Basic Packages
import numpy as np                 # Numpy
import pandas as pd                 # Zur Arbeit mit dataframes

# Import Visualization Packages
from collections import Counter     # Um worte zu zählen
import matplotlib.pyplot as plt   # Für Visualisierung
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #Wordcloud erstellen

# Import NLP-Packages
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

pd.options.mode.chained_assignment = None
import spacy 

# Import Methods from the GitHub 
import Sentiment_Methods

### 1.2 Datensatz einlesen

In diesem Abschnitten werden die JSON-Dateien mit den Reden aus der 19. und 20. Wahlperiode eingelesen. 

In [3]:
# Einlesen der JSON-Datei "speeches_19.jsonl" 
# Legislaturperiode 2019 (24.10.2017-26.09.21)
df19 = pd.read_json(r"C:\Users\paula\OneDrive\Uni\2. Lernpools Master\3. Semester\Soziologie\Data_Speeches\speeches_19.jsonl", lines=True)
df19['date'] = pd.to_datetime(df19['date'])
df19.sort_values(by='date')


# Einlesen der JSON-Datei "speeches_20.jsonl"
# Legislaturperiode 2020 (24.10.2017-26.09.21)
df20 = pd.read_json(r"C:\Users\paula\OneDrive\Uni\2. Lernpools Master\3. Semester\Soziologie\Data_Speeches\speeches_20.jsonl", lines=True)
df20['date'] = pd.to_datetime(df20['date'])
df20.sort_values(by='date')

,text,id,period,date,name,party,redner_id,discussion_title
8620,"Frau Präsidentin, ich nehme die Wahl an und fr...",ID20101400,18,2021-10-26,Petra Pau,DIE LINKE,11003206,Tagesordnungspunkt 6 2021-10-26
8607,Sehr geehrter Herr Alterspräsident! So muss ic...,ID20100100,18,2021-10-26,Gabriele Katzmarek,SPD,11004325,Tagesordnungspunkt 2 2021-10-26
8608,Herr Präsident! Liebe Kolleginnen und Kollegen...,ID20100200,18,2021-10-26,Stefan Müller,CDU/CSU,11003597,Tagesordnungspunkt 2 2021-10-26
8609,Sehr geehrter Herr Alterspräsident Wolfgang Sc...,ID20100300,18,2021-10-26,Britta Haßelmann,BÜNDNIS 90/DIE GRÜNEN,11003764,Tagesordnungspunkt 2 2021-10-26
8610,Sehr geehrter Herr Präsident! Meine lieben Kol...,ID20100400,18,2021-10-26,Marco Buschmann,FDP,11004023,Tagesordnungspunkt 2 2021-10-26
...,...,...,...,...,...,...,...,...
9145,Vielen Dank. – Frau Präsidentin! Meine sehr ve...,ID209804700,18,2023-04-21,Caren Lay,DIE LINKE,11004088,Tagesordnungspunkt 24 2023-04-21
9146,Sehr geehrte Frau Präsidentin! Werte Kolleginn...,ID209804800,18,2023-04-21,Daniel Föst,FDP,11004716,Tagesordnungspunkt 24 2023-04-21
9147,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...,ID209804900,18,2023-04-21,Petra Nicolaisen,CDU/CSU,11004841,Tagesordnungspunkt 24 2023-04-21
9138,Frau Präsidentin! Herr Staatssekretär! Meine l...,ID209804000,18,2023-04-21,Mechthilde Wittmann,CDU/CSU,11005261,Tagesordnungspunkt 13 2023-04-21


### 1.3 Datensätze bearbeiten
In diesem Schritt werden die eingelesenen Reden mithilfe von Schlüsselwörtern und einem zeitlichen Filter auf relevante Inhalte beschränkt, da uns nur die Reden interessieren, die sich im Kontext von Corona bewegen. 

Zunächst werden die Schlüsselwörter definiert, wobei hier 'Covid-19' & Synonyme als Hauptschlüsselwort verwendet werden. Anschließend werden die politischen Reden in einem DataFrame gefiltert. 


In [5]:
# Filtern df19 nach Datum
# Startpunkt Auftreten des 1. Falls in DE

start_date = pd.to_datetime('2020-01-27')
df19_filtered = df19[df19['date'] >= start_date]

In [14]:
# Festlegung der Keywords nach den gesucht werden soll.
# Unter Verwendung des Moduls Sentiment_Methods wird dabei innerhalb der Reden nach Keywords und Synonymen gesucht
keywords = ['Covid-19']

# Aufruf der Methode, um die Reden aus der 19. Periode zu erhalten, die entsprechende Keywords haben.
df19_Covid = Sentiment_Methods.filter_dataframe_by_keywords_with_synonyms(df19_filtered, keywords)
print(f"Anzahl der Reden in der 19. Wahlperiode mit dem Stichwort oder Synonym 'Covid-19': {len(df19_Covid)}")

# Aufruf der Methode, um die Reden aus der 20. Periode zu erhalten, die entsprechende Keywords haben.
df20_Covid = Sentiment_Methods.filter_dataframe_by_keywords_with_synonyms(df20, keywords)


Anzahl der Reden in der 19. Wahlperiode mit dem Stichwort oder Synonym 'Covid-19': 203


In [10]:

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def count_occurrences_of_keywords(df, keywords):
    def count_occurrences(text):
        word_tokens = word_tokenize(text.lower())
        keyword_occurrences = [1 for word in word_tokens if word in keywords]
        synonym_occurrences = [1 for word in word_tokens for keyword in keywords if word in get_synonyms(keyword)]
        return sum(keyword_occurrences) + sum(synonym_occurrences)
    
    df['occurrences'] = df['text'].apply(count_occurrences)
    return df


# List of keywords (including 'corona')
keywords = ['covid-19']

# Count occurrences of keywords and synonyms in each cell of 'text'
df_with_occurrences = count_occurrences_of_keywords(df20_Covid, keywords)

# Display the DataFrame with the occurrence count
print(df_with_occurrences)



                                                    text           id  period  \
216    Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...  ID203110500      18   
223    Vielen Dank. – Frau Präsidentin! Die einrichtu...  ID203111200      18   
224    Sehr geehrte Frau Präsidentin! Werte Kolleginn...  ID203111300      18   
226    Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...  ID203111500      18   
366    Frau Präsidentin! Meine Damen und Herren! Von ...  ID207910600      18   
849    Vielen Dank, Herr Präsident. – Liebe Kolleginn...  ID204610600      18   
1032   Frau Präsidentin! Liebe Kolleginnen und Kolleg...  ID207317600      18   
1493   Sehr geehrte Frau Präsidentin! Meine Damen und...   ID20401000      18   
2319   Frau Präsidentin! Liebe Kolleginnen und Kolleg...  ID201307500      18   
2325   Sehr geehrte Frau Präsidentin! Liebe Kolleginn...  ID201308100      18   
2335   Sehr geehrte Frau Präsidentin! Als ungeimpfter...  ID201309100      18   
2336   Sehr geehrte Frau Prä

### 1.4 Textdaten bereinigen
Im nächsten Schritt werden Methoden aufgerufen, um das Sentiment der Reden auf Satzebene analysieren zu können. Die Methode tokenize_and_split_sentences(df) nimmt den DataFrame mit den Reden entgegen, tokenisiert die darin enthaltenen Sätze mithilfe von NLTK und teilt sie in einzelne Sätze des DataFrames auf, wodurch ein neues DataFrame erstellt wird, das sowohl die ursprünglichen Zeilendaten als auch die tokenisierten Sätze enthält. Dies ist hilfreich, um Textdaten auf Satzebene zu analysieren oder weiter zu verarbeiten.

In [11]:
#Methodenaufruf,tokenize_and_split_sentences(df)
df19_Covid_Split = Sentiment_Methods.tokenize_and_split_sentences (df19_Covid)
df20_Covid_Split = Sentiment_Methods.tokenize_and_split_sentences (df20_Covid)

In [12]:
# Download der Deutschen Stoppwortliste
nltk.download('stopwords')

# Festlegen einer eigenen Stoppwortliste
my_stopwords = ['ministerin', 'minister', 'kollege', 'kollegin', 'präsidentin', 'kolleginnen', 'kollegen', 'frau', 'herr', 'mann', 'geehrte', 'geehrter', 'liebe', 'lieber', 'damen', 'herren', 'präsident', 'vielen dank','dank','redner','rednerin']

# Methodenaufruf clean_text zur Textverarbeitung und einfacheren Erstellung von n-grammen
df19_Covid_cleaned = Sentiment_Methods.clean_text(df19_Covid_Split, my_stopwords)

df20_Covid_cleaned = Sentiment_Methods.clean_text(df20_Covid_Split, my_stopwords)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,satz,id,period,date,name,party,redner_id,discussion_title,text,tokenized_text,tokens,cleaned_text
2,Die einrichtungsbezogene Impfpflicht mit der e...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[einrichtungsbezogene, Impfpflicht, erklärten,...",einrichtungsbezogene Impfpflicht erklärten Zie...
3,In der gestrigen Sitzung des Gesundheitsaussch...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[gestrigen, Sitzung, Gesundheitsausschusses, s...",gestrigen Sitzung Gesundheitsausschusses stell...
4,Vom Ministerium und vom RKI gab es nur ein gan...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[Ministerium, RKI, gab, ganz, kurzes, Nein, An...",Ministerium RKI gab ganz kurzes Nein Antwort
5,Vom Paul-Ehrlich-Institut erhielt ich überhaup...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[Paul-Ehrlich-Institut, erhielt, überhaupt, An...",Paul-Ehrlich-Institut erhielt überhaupt Antwort
6,Professor Wieler äußerte in der nächsten Runde...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[Professor, Wieler, äußerte, nächsten, Runde, ...",Professor Wieler äußerte nächsten Runde Satz g...
7,Eine Modellierung?,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...",[Modellierung],Modellierung
8,Da draußen gibt es doch echte Menschen in den ...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[draußen, gibt, echte, Menschen, medizinischen...",draußen gibt echte Menschen medizinischen Beru...
9,"Deshalb müsste es doch möglich sein, dass in V...",ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[Deshalb, müsste, möglich, Vorbereitung, Geset...",Deshalb müsste möglich Vorbereitung Gesetzes e...
10,Sie ahnen es vielleicht schon: Diesen wissensc...,ID203110500,18,2022-04-28,Kay-Uwe Ziegler,AfD,11005265,Tagesordnungspunkt 17 2022-04-28,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,"[Sehr geehrte Frau Präsidentin!, Sehr geehrte ...","[ahnen, vielleicht, schon, wissenschaftlichen,...",ahnen vielleicht schon wissenschaftlichen Nach...


## 2. Deskriptive Analyse der Reden
Im nachfolgenden soll eine kurze deskriptive Beschreibung der gefundenen Datensätze stattfinden, dazu wurden im Modul Sentiment_Methods eigene Methoden definiert, die n-Gramme und Wordclouds ausgeben. 

### 2.1 N-Gramme
N-Gramme sind nacheinander auftretende Gruppen von Wörtern oder Zeichen in einem Text. N-Gramme ermöglichen es, Kontext und Bedeutung in Texten zu erfassen, indem sie häufige Wortkombinationen und Phrasen identifizieren. Im folgenden werden die n-gramme für die beiden gefilterten Dataframes erstellt.

In [15]:
#Ausgabe der nGrams 
ngram_19= Sentiment_Methods.plot_most_frequent_ngrams (df19_Covid_cleaned)
ngram_20 = Sentiment_Methods.plot_most_frequent_ngrams (df20_Covid_cleaned)

NameError: name 'df19_Covid_cleaned' is not defined

## 3. Identifizierung des Sentiments unter Verwendung von Transformer-Modellen
Die Identifizierung des Sentiments während der Corona Pandemie erfolgt unter Verwendung von Transformer-Modellen. Durch die Anwendung eines vortrainierten Transformer-Modells German Sentiment Classification with Bert auf den gegebenen Text. 

@InProceedings{guhr-EtAl:2020:LREC,
  author    = {Guhr, Oliver  and  Schumann, Anne-Kathrin  and  Bahrmann, Frank  and  Böhme, Hans Joachim},
  title     = {Training a Broad-Coverage German Sentiment Classification Model for Dialog Systems},
  booktitle      = {Proceedings of The 12th Language Resources and Evaluation Conference},
  month          = {May},
  year           = {2020},
  address        = {Marseille, France},
  publisher      = {European Language Resources Association},
  pages     = {1620--1625},
  url       = {https://www.aclweb.org/anthology/2020.lrec-1.202}
}


### 3.1 Modellaufruf
Wir beginnen mit dem Methodenaufruf aus dem Modul Sentiment_methods. Dieser verwendet eine Pipeline, um das German Sentiment Bert-Modell für die Sentiment-Analyse zu nutzen. Dabei nutzen wir keine eigenen Text-Preprocessing-Schritte, da die Methode diese automatisch durchführt. Wir wenden das Modell auf Satzebene an, um das Sentiment zu analysieren.

In [16]:
# Aufruf Transformer Modell unter Verwendung einer eigenen Methode
import torchvision

#Original Text - Modell Periode 19
modell_original19 = Sentiment_Methods.sentiment_analysis(df19_Covid_cleaned, 'satz')

#Original Text - Modell Periode 19
modell_original20 = Sentiment_Methods.sentiment_analysis(df20_Covid_cleaned, 'satz')


NameError: name 'df19_Covid_cleaned' is not defined

Um die Qualität der Sentiment-Analyse zu vergleichen, werden die Transformer-Modelle auch auf Satzebene mit den eigenen preprozessierten Sätzen angewendet. Dies gewährleistet einen fairen und aussagekräftigen Vergleich der Analyseergebnisse.

In [19]:
# Modellaufruf mit preprocessed Text Periode 19
modell_processed19 = Sentiment_Methods.sentiment_analysis(df19_Covid_cleaned, 'cleaned_text')

# Modellaufruf mit preprocessed Text Periode 20
modell_processed20 = Sentiment_Methods.sentiment_analysis(df20_Covid_cleaned, 'cleaned_text')

### 3.2 Deskriptive Darstellung der Sentimentanalyse

In [27]:
Sentiment_Methods.plot_sentiment_analysis(modell_original19, modell_processed19)

In [32]:
Sentiment_Methods.plot_sentiment_wordclouds (modell_cleaned19)
Sentiment_Methods.plot_sentiment_wordclouds (modell_processed19)

NameError: name 'modell_cleaned19' is not defined